# Hate Speech Detection Using LSTM in PyTorch

In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('labeled_data.csv')

df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [47]:
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

In [48]:
text = []
clas = []

text = df['tweet'].tolist()
clas = df['class'].tolist()

In [49]:
df = pd.DataFrame({'tweet': text, 'class': clas})

In [50]:
print(df.isnull().sum())

tweet    0
class    0
dtype: int64


In [51]:
df['tweet'] = df['tweet'].apply(lambda x:x.lower())

In [52]:
punctuation_signs = list("?:!.,;|")
remove_usernames = r'@([^\s]+)'

for punct_sign in punctuation_signs:
    df['tweet'] = df['tweet'].str.replace(punct_sign, '')

df['tweet'] = df['tweet'].str.replace(remove_usernames, '')

<ipython-input-52-0fc482e0f446>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['tweet'] = df['tweet'].str.replace(punct_sign, '')
<ipython-input-52-0fc482e0f446>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df['tweet'].str.replace(remove_usernames, '')


In [53]:
df['tweet'] = df['tweet'].apply(lambda x: x.replace('\n', ' '))
df['tweet'] = df['tweet'].apply(lambda x: x.replace('\t', ' '))
df['tweet'] = df['tweet'].str.replace("    ", " ")
df['tweet'] = df['tweet'].str.replace('"', '')
df['tweet'] = df['tweet'].str.replace("!", "")
df['tweet'] = df['tweet'].str.replace("'s", "")
df['tweet'] = df['tweet'].str.replace("'", "")
df['tweet'] = df['tweet'].str.replace("`", "")
df['tweet'] = df['tweet'].str.replace("https", "")
df['tweet'] = df['tweet'].str.replace("http", "")
df['tweet'] = df['tweet'].str.replace("rt", "")
df['tweet'] = df['tweet'].str.replace("∼", "")

In [54]:
df

,tweet,class
0,as a woman you shouldnt complain about clea...,2
1,boy dats coldtyga dwn bad for cuffin dat ho...,1
2,dawg you ever fuck a bitch and she sta to...,1
3,she look like a tranny,1
4,the shit you hear about me might be true or...,1
...,...,...
24778,you a muthaf***in lie &#8220 right his tl is...,1
24779,youve gone and broke the wrong hea baby and dr...,2
24780,young buck wanna eat dat nigguh like i aint fu...,1
24781,youu got wild bitches tellin you lies,1


In [55]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    df['tweet'] = df['tweet'].str.replace(regex_stopword, '')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-55-969333b8111c>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df['tweet'].str.replace(regex_stopword, '')


In [56]:
df

,tweet,class
0,woman shouldnt complain cleaning hous...,2
1,boy dats coldtyga dwn bad cuffin dat hoe ...,1
2,dawg ever fuck bitch sta cry confu...,1
3,look like tranny,1
4,shit hear might true might faker ...,1
...,...,...
24778,muthaf*** lie &#8220 right tl trash &#82...,1
24779,youve gone broke wrong hea baby drove redn...,2
24780,young buck wanna eat dat nigguh like aint fuc...,1
24781,youu got wild bitches tellin lies,1


In [57]:
cv = CountVectorizer(max_features = 125)
X = cv.fit_transform(df['tweet']).toarray()
y = df['class']

In [58]:
print(cv.get_feature_names_out())

['128175' '128514' '128530' '128553' '128557' '65039' '8220' '8221' '8230'
 'aint' 'always' 'amp' 'ass' 'back' 'bad' 'better' 'big' 'bird' 'bitch'
 'bitches' 'black' 'bout' 'call' 'cant' 'cause' 'charlie' 'come' 'cunt'
 'da' 'damn' 'dat' 'day' 'dick' 'dont' 'dumb' 'eat' 'even' 'ever' 'every'
 'fag' 'faggot' 'fat' 'fuck' 'fuckin' 'fucking' 'get' 'getting' 'ghetto'
 'girl' 'girls' 'give' 'go' 'going' 'gonna' 'good' 'got' 'gotta' 'gt'
 'hate' 'hit' 'hoe' 'hoes' 'ill' 'im' 'keep' 'know' 'let' 'life' 'like'
 'lil' 'little' 'lmao' 'lol' 'look' 'love' 'make' 'man' 'money' 'much'
 'need' 'never' 'new' 'nigga' 'niggah' 'niggas' 'nigger' 'one' 'people'
 'pussy' 'put' 'real' 'really' 'retarded' 'right' 'said' 'say' 'see'
 'shit' 'still' 'stop' 'stupid' 'take' 'talk' 'talking' 'tco' 'tell'
 'think' 'tho' 'time' 'trash' 'twitter' 'ugly' 'ur' 'wanna' 'want' 'way'
 'white' 'wit' 'would' 'ya' 'yall' 'yankees' 'yellow' 'yo' 'youre']


In [59]:
X[24781], y

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0        2
 1        1
 2        1
 3        1
 4        1
         ..
 24778    1
 24779    2
 24780    1
 24781    1
 24782    2
 Name: class, Length: 24783, dtype: int64)

In [60]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt


print(torch.__version__)

# Setting up device agnostic
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

2.1.0+cu121
cuda


## Spliting Datasets

In [61]:
X.dtype, y.dtype

(dtype('int64'), dtype('int64'))

In [62]:
# Conversion from int64 to long
X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y)

X, y

(tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 1, 0, 0]]),
 tensor([2, 1, 1,  ..., 1, 1, 2]))

In [63]:
train_split = int(0.8* len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train),len(y_train), len(X_test), len(y_test)

(19826, 19826, 4957, 4957)

In [64]:


y_train.dtype, y_test.dtype

(torch.int64, torch.int64)

## Neural Network Model

In [65]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
    super(LSTMModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.dropout = nn.Dropout(0.2)
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    embedded = self.embedding(x)
    embedded = self.dropout(embedded)
    lstm_out, _ = self.lstm(embedded)
    lstm_out = lstm_out[:, -1, :]
    fc_out = self.fc(lstm_out)
    output = self.softmax(fc_out)
    return output

In [66]:
vocab_size = 232337
embedding_dim = 100
hidden_dim = 20
output_dim = 3

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model

LSTMModel(
  (embedding): Embedding(232337, 100)
  (lstm): LSTM(100, 20, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=20, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [67]:
next(model.parameters()).device

device(type='cpu')

In [68]:
# Set model usage to CUDA
model.to(device)
next(model.parameters()).device

device(type='cuda', index=0)

In [69]:
# Loss function and optimizers
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [70]:
epochs = 50
batch_size = 64

# Placing data on CUDA
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

# Train loop
for epoch in range(epochs):
  model.train()
  y_pred = model(X_train)
  loss = loss_fn(y_pred, y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # Testing
  model.eval()
  with torch.inference_mode():
    test_pred = model(X_test)
    test_loss = loss_fn(test_pred, y_test)

  if epoch % 10 == 0 or epoch == epochs-1:
    print(f'Epoch: {epoch} | Train Loss: {loss:.4f} | Test Loss: {test_loss:.4f}')

Epoch: 0 | Train Loss: 1.0351 | Test Loss: 0.9141
Epoch: 10 | Train Loss: 0.7883 | Test Loss: 0.7647
Epoch: 20 | Train Loss: 0.7829 | Test Loss: 0.7603
Epoch: 30 | Train Loss: 0.7822 | Test Loss: 0.7596
Epoch: 40 | Train Loss: 0.7820 | Test Loss: 0.7594
Epoch: 49 | Train Loss: 0.7819 | Test Loss: 0.7593


## Saving model

In [72]:
from pathlib import Path

model_path = Path('models')
model_path.mkdir(parents = True, exist_ok =True)

model_name = '00_LSTM_HateSpeech.pth'
model_save_path = model_path / model_name

print(f"Saving model to: {model_save_path}")
torch.save(obj = model.state_dict(),
           f = model_save_path)
print('Saved Successfully')

Saving model to: models/00_LSTM_HateSpeech.pth
Saved Successfully


In [75]:
loaded_model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
loaded_model.load_state_dict(torch.load(model_save_path))

# Transfer loaded model to CUDA
loaded_model.to(device)

print(f"Loaded model:\n{loaded_model}")
print(f"Model on device:\n{next(loaded_model.parameters()).device}")

Loaded model:
LSTMModel(
  (embedding): Embedding(232337, 100)
  (lstm): LSTM(100, 20, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=20, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)
Model on device:
cuda:0


In [77]:
model.eval()

with torch.inference_mode():
  y_preds = model(X_test)

y_preds

tensor([[4.9096e-04, 9.9905e-01, 4.6303e-04],
        [4.9096e-04, 9.9905e-01, 4.6303e-04],
        [4.9096e-04, 9.9905e-01, 4.6303e-04],
        ...,
        [4.9096e-04, 9.9905e-01, 4.6303e-04],
        [4.9096e-04, 9.9905e-01, 4.6303e-04],
        [4.8766e-04, 9.9905e-01, 4.6009e-04]], device='cuda:0')

In [79]:
# Evaluate loaded model
loaded_model.eval()
with torch.inference_mode():
  loaded_model_preds = loaded_model(X_test)

y_preds == loaded_model_preds

tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        ...,
        [True, True, True],
        [True, True, True],
        [True, True, True]], device='cuda:0')